In [2]:
!pip install pyjwt

  Using cached PyJWT-2.10.1-py3-none-any.whl.metadata (4.0 kB)
Using cached PyJWT-2.10.1-py3-none-any.whl (22 kB)


In [5]:
import jwt
import requests

In [ ]:
import json
import time
import jwt  # PyJWT library (install with `pip install PyJWT`)
import requests

# Path to your Google Cloud JSON key
KEY_PATH = "google_api.json"

# Load JSON credentials
with open(KEY_PATH) as f:
    creds = json.load(f)

# Google OAuth 2.0 Token URL
GOOGLE_OAUTH2_TOKEN_URL = "https://oauth2.googleapis.com/token"

def get_access_token():
    """Generate an OAuth 2.0 access token manually using service account JSON."""
    
    # Define JWT claim set
    issued_at = int(time.time())
    expiration_time = issued_at + 3600  # Token valid for 1 hour

    payload = {
        "iss": creds["client_email"],  # Issuer (Service Account Email)
        "scope": "https://www.googleapis.com/auth/cloud-translation",  # Translation API scope
        "aud": GOOGLE_OAUTH2_TOKEN_URL,
        "iat": issued_at,  # Issued At
        "exp": expiration_time  # Expiry
    }

    # Create a JWT signed with the private key
    private_key = creds["private_key"]
    jwt_token = jwt.encode(payload, private_key, algorithm="RS256")

    # Exchange JWT for OAuth 2.0 access token
    token_request_data = {
        "grant_type": "urn:ietf:params:oauth:grant-type:jwt-bearer",
        "assertion": jwt_token
    }

    response = requests.post(GOOGLE_OAUTH2_TOKEN_URL, data=token_request_data)
    
    if response.status_code == 200:
        return response.json()["access_token"]
    else:
        raise Exception(f"Error obtaining access token: {response.text}")

def translate(text, target_lang="es"):
    """Use Google Cloud Translation API with manually obtained access token."""
    
    token = get_access_token()
    url = "https://translation.googleapis.com/language/translate/v2"

    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }

    payload = {
        "q": text,
        "target": target_lang
    }

    response = requests.post(url, headers=headers, json=payload)

    if response.status_code == 200:
        return response.json()["data"]["translations"][0]["translatedText"]
    else:
        return f"Error: {response.text}"

# Example Usage
translated_text = translate("Hello, how are you?", target_lang="pl")
print(translated_text)  # Output: Bonjour, comment ça va ?


Witam, jak się masz?
